In [4]:
import cv2
import torch
from ultralytics import YOLO
from sort import *
import math

In [50]:
cap = cv2.VideoCapture(r"C:\Users\aashutosh kumar\Downloads\CARS.mp4")
model = YOLO("yolov8l.pt")

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

total_counts = []
mask = cv2.imread(r"C:\Projects\Car-Counter\CAR-COUNTER\MASK1.png")
mask = cv2.resize(mask, (640, 360))  # Resize mask to match the frame size
limits = [100, 250, 600, 250]

tracker = Sort(max_age = 20,min_hits = 2,iou_threshold = 0.3)

while True:
    r, frame = cap.read()

    if r:
        frame_Rgn = cv2.bitwise_and(frame, mask)
        results = model(frame_Rgn, stream=True)

        detections = np.empty((0, 5))

        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                class_id = int(box.cls[0])
                confidence = math.ceil((box.conf[0] * 100)) / 100
                class_name = classNames[class_id]

                if class_name == "car" and confidence > 0.4:
                    curr_array = np.array([x1, y1, x2, y2, confidence])
                    detections = np.vstack((detections, curr_array))

        cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)
        tracking_res = tracker.update(detections)

        for results in tracking_res:
            x1, y1, x2, y2, id = results
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (150, 200, 0), 3, 1)
            label = f"conf {confidence}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150, 200, 0), 1)

            w = x2 - x1
            h = y2 - y1
            cx = x1 + w // 2
            cy = y1 + h // 2

            cv2.circle(frame, (cx, cy), 3, (255, 0, 0), cv2.FILLED)
            if limits[0] < cx < limits[2] and limits[1] - 2 < cy < limits[1] + 2:
                if id not in total_counts:
                    total_counts.append(id)
                    cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)

        cv2.putText(frame,str(len(total_counts)),(25,70),cv2.FONT_HERSHEY_PLAIN,5,(15,241,249),8)

        # cv2.imshow("ROI", frame_Rgn)
        cv2.imshow("Cars", frame)

        if cv2.waitKey(25) & 0xff == ord("p"):
            break

cv2.destroyAllWindows()
cap.release()



0: 384x640 5 cars, 1 bus, 32.6ms
Speed: 2.5ms preprocess, 32.6ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 30.2ms
Speed: 1.0ms preprocess, 30.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 30.3ms
Speed: 1.0ms preprocess, 30.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 31.6ms
Speed: 0.0ms preprocess, 31.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 30.6ms
Speed: 1.3ms preprocess, 30.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 28.5ms
Speed: 2.3ms preprocess, 28.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 28.6ms
Speed: 2.0ms preprocess, 28.6ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 29.5ms
Speed: 1.3ms preprocess, 29.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



In [25]:
total_counts

[]